In [1]:
%load_ext rpy2.ipython

In [ ]:
%R
# 2-Class Model for normal mean model
# Conjugate priors (mu~N(m0,1/t0), sigma2<-Inv-chi(a, b))
# FOR MORE INFO, SEE _FINITE MIXTURE AND MARKOV SWITCHING MODELS_
#	BY SILVIA FRUWIRTH-SCHNATTER (2006, SPRINGER PRESS) 
# June 2010
###################################
library(mixtools)

########
# Data #
########
set.seed(250)
n<-100
pi<-c(.5,.5)  # Two-Component Mixture
mu<-c(-1,1)	  # Normal means
s2<-c(1,2)	  # Variances
y<-rnormmix(n,pi,mu,sqrt(s2))
plot(density(y),type="l",col="darkgreen",lwd=2)

####################
# Prior Hyperparms #
####################
m0<-0		# Prior mean of mu
t0<-.01	# Prior precision of mu
a<-b<-.001  # Hyperparms for error precisions
l<-d<-4	# Hyperparms for beta distrbn  (use 4 to avoid zero groups -- see F-S, 2006)

#########
# Inits #
#########
tau1<-tau2<-1
mu1<-mu2<-0
c<-rbinom(n,1,.5)+1
n1<-length(c[c==1])

#################
# Store Results #
#################
nsim<-5000
Tau1<-Tau2<-Mu1<-Mu2<-Sigma1<-Sigma2<-rep(0,nsim)
C<-matrix(0,nsim,n)

#########
# Gibbs #
#########
for (i in 1:nsim) {

  # Update latent class hyperparms
    pi1<-rbeta(1,n1+4,n-n1+4)			# Post prob of class 1
    pi2<-1-pi1

 # Update C (latent class indicator) 
    p2<-pi2*dnorm(y,mu2,sqrt(1/tau2))/(pi1*dnorm(y,mu1,sqrt(1/tau1))+pi2*dnorm(y,mu2,sqrt(1/tau2)))  # Posterior prob of belonging to class 2
  
   C[i,]<-c<-rbinom(n,1,p2)+1		# Class Indicator
   n1<-length(c[c==1])
   n2<-n-n1

 # Update mu1
   v1<-1/(tau1*n1+ t0)
   m1<-v1*(tau1*sum(y[c==1])+t0*m0)
   #mu1<- Mu1[i]<-rnorm(1,m1,sqrt(v1))  # Note unconstrained induces 
						    # lab switching -- Try it!
   mu1<-Mu1[i]<-qnorm(runif(1,pnorm(mu2,m1,sqrt(v1)),1),m1,sqrt(v1)) # Order constraint avoids
											   # label switch but may force two
											   # classes when 1 is enough?

 # Update mu2 contrained to be < mu1 (to avoid label switching)
   v2<-1/(tau2*n2+ t0)
   m2<-v2*(tau2*sum(y[c==2])+t0*m0)
   #mu2<-Mu2[i]<- rnorm(1,m2,sqrt(v2))  # Note unconstrained induces 
						    # lab switching -- Try it!
   mu2<-Mu2[i]<-qnorm(runif(1,0,pnorm(mu1,m2,sqrt(v2))),m2,sqrt(v2))

  tau1<-Tau1[i]<-rgamma(1,a+n1/2,b+sum((y[c==1]-mu1)^2)/2)
  tau2<-Tau2[i]<-rgamma(1,a+n2/2,b+sum((y[c==2]-mu2)^2)/2)

  Sigma1[i]<-1/tau1
  Sigma2[i]<-1/tau2

 if (i%%100==0) print(i)
}
mean(Mu1[501:nsim])
mean(Mu2[501:nsim])
mean(sqrt(Sigma1[501:nsim]))
mean(sqrt(Sigma2[501:nsim]))
plot(501:nsim,Mu1[501:nsim],type="l",col="lightgreen")
abline(h=mean(Mu1[501:nsim]))

In [3]:
#!gist -p mixture_model_using_gibbs_sampling.ipynb

In [5]:
!gist -u https://gist.github.com/7afa69614e184c2fc0f4 mixture_model_using_gibbs_sampling.ipynb

https://gist.github.com/7afa69614e184c2fc0f4
